In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
# import stellargraph as sg

In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [3]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import os
import networkx as nx
import numpy as np
import pandas as pd
import random
# from stellargraph import datasets
# from IPython.display import display, HTML
# from stellargraph.data import UniformRandomMetaPathWalk

%matplotlib inline

In [29]:
path = '/datasets/dsc180a-wi20-public/Malware/group_data/group_01/pipeline_output'

In [5]:
A_tr = sparse.load_npz(os.path.join(path, 'A_reduced_tr.npz'))
B_tr = sparse.load_npz(os.path.join(path, 'B_reduced_tr.npz')).tocsr()
P_tr = sparse.load_npz(os.path.join(path, 'P_reduced_tr.npz')).tocsr()
A_tr_csr = A_tr
A_tr_csc = A_tr.tocsc(copy=True)

# Create metapath

In [6]:
import m2v
%load_ext autoreload
%autoreload 2

In [7]:
metapaths = list('APA')
model = m2v.Metapath2Vec(A_tr, B_tr, P_tr)

In [15]:
%%time
# test one iteration of metapath2vec
for i in range(1):
    path = []
    print(model.metapath2vec(metapaths, path, 0))
# print(path)

['app_0', 'api_114', 'api_463', 'app_1274']
CPU times: user 1.99 ms, sys: 0 ns, total: 1.99 ms
Wall time: 1.71 ms


# Create Corpus

In [19]:
%%time
fp = '/datasets/dsc180a-wi20-public/Malware/group_data/group_01/metapath_corpus'
meta = 'AA'
corpus_name = 'meta_%s.cor'%meta
metapaths = list('%s'%meta)

model = m2v.Metapath2Vec(A_tr, B_tr, P_tr)

f = open(os.path.join(fp, corpus_name), "w")

for appNum in range(A_tr.shape[0]):
    for times in range(1000):
        path = []
        f.write(' '.join(model.metapath2vec(metapaths, path, appNum)) + '\n')
    
f.close()

CPU times: user 6min 15s, sys: 567 ms, total: 6min 15s
Wall time: 6min 15s


# Train word embeddings

In [24]:
from gensim import utils

class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = os.path.join(fp, 'meta_AA.cor')
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield line.strip().split(' ')

In [25]:
%%time
import gensim.models

sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences, min_count=1, size=200, window=2)

CPU times: user 1min 24s, sys: 1.16 s, total: 1min 25s
Wall time: 41.3 s


In [28]:
path

['app_1334', 'api_166', 'app_418']

In [32]:
label = pd.read_csv(os.path.join(path, 'meta_tr.csv'))

In [33]:
def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = []  # positions in vector space
    labels = []  # keep track of words to label our data again later
    for word in model.wv.vocab:
        if 'app' in word:
            if (label.iloc[int(word.split('_')[1]), 1] == 'class1'):
                labels.append(1)
            else:
                labels.append(0)

            vectors.append(model.wv[word])
            # labels.append(word)

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)
    # reduce using t-SNE
    vectors = np.asarray(vectors)
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='markers',
                       text=labels, marker=dict(size=5, color=labels))

    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)
    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))
    plt.show()
    
def cosine(u, v):
    return np.dot(u, v)/(np.linalg.norm(u)* np.linalg.norm(v))

In [34]:
x_vals, y_vals, labels = reduce_dimensions(model)
plot_with_plotly(x_vals, y_vals, labels)

In [39]:
label.iloc[583, 1]

'class1'

In [117]:
model.wv.most_similar('app_0')

[('app_15', 0.6511681079864502),
 ('app_577', 0.6464868783950806),
 ('app_49', 0.6373293995857239),
 ('app_583', 0.6354970335960388),
 ('app_148', 0.6100234985351562),
 ('app_54', 0.5989775657653809),
 ('app_491', 0.5841787457466125),
 ('app_302', 0.5799149870872498),
 ('app_264', 0.5792275667190552),
 ('app_534', 0.569420576095581)]